<a href="https://colab.research.google.com/github/fboldt/aulasml/blob/master/aula08a_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip titanic.zip

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [3]:
import pandas as pd
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.dtypes

,0
PassengerId,int64
Survived,int64
Pclass,int64
Name,object
Sex,object
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64


In [5]:
y = df['Survived']
X = df.drop('Survived', axis=1)

In [6]:
len(y)

891

In [9]:
for col in X.columns:
  print(f"{col:>12} {len(set(X[col])):4} {X[col].dtype}")

 PassengerId  891 int64
      Pclass    3 int64
        Name  891 object
         Sex    2 object
         Age  265 float64
       SibSp    7 int64
       Parch    7 int64
      Ticket  681 object
        Fare  248 float64
       Cabin  148 object
    Embarked    4 object


In [10]:
caracteristicas_indesejadas = ['PassengerId', 'Name', 'Ticket', 'Cabin']
Xdrop = X.drop(caracteristicas_indesejadas, axis=1)
Xdrop.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [11]:
Xnum = Xdrop.select_dtypes('number')
Xnum.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [13]:
for col in Xnum.columns:
  print(f"{col:>12} {Xnum[col].isnull().sum():2}")

      Pclass  0
         Age 177
       SibSp  0
       Parch  0
        Fare  0


In [14]:
Xnum[-5:]

,Pclass,Age,SibSp,Parch,Fare
886,2,27.0,0,0,13.00
887,1,19.0,0,0,30.00
888,3,NaN,1,2,23.45
889,1,26.0,0,0,30.00
890,3,32.0,0,0,7.75


In [15]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
XnumTratado = imputer.fit_transform(Xnum)
XnumTratado[-5:]

array([[ 2.  , 27.  ,  0.  ,  0.  , 13.  ],
       [ 1.  , 19.  ,  0.  ,  0.  , 30.  ],
       [ 3.  , 28.  ,  1.  ,  2.  , 23.45],
       [ 1.  , 26.  ,  0.  ,  0.  , 30.  ],
       [ 3.  , 32.  ,  0.  ,  0.  ,  7.75]])

In [16]:
Xcat = Xdrop.select_dtypes('object')
Xcat.columns

Index(['Sex', 'Embarked'], dtype='object')

In [19]:
for col in Xcat.columns:
  print(f"{col:>12} {Xcat[col].isnull().sum():2}")

         Sex  0
    Embarked  2


In [20]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
XcatTratado = imputer.fit_transform(Xcat)
XcatTratado[-5:]

array([['male', 'S'],
       ['female', 'S'],
       ['female', 'S'],
       ['male', 'C'],
       ['male', 'Q']], dtype=object)

In [21]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
XcatTratadoHot = encoder.fit_transform(XcatTratado)
XcatTratadoHot

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1782 stored elements and shape (891, 5)>

In [22]:
import numpy as np
Xtratado = np.c_[XnumTratado, XcatTratadoHot.toarray()]
Xtratado.shape

(891, 10)

In [24]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosDesejados(BaseEstimator, TransformerMixin):
  def __init__(self):
    self.colunas_indesejadas = ['PassengerId', 'Name', 'Ticket', 'Cabin']
  def fit(self, X, y=None):
    return self
  def transform(self, X, y=None):
    return X.drop(self.colunas_indesejadas, axis=1)

atributos_desejados = AtributosDesejados()
Xdrop = atributos_desejados.fit_transform(X)
Xdrop.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [25]:
class AtributosNumericos(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self
  def transform(self, X, y=None):
    return X.select_dtypes(include='number')

atributos_numericos = AtributosNumericos()
Xnum = atributos_numericos.fit_transform(Xdrop)
Xnum.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


In [26]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

pipenum = Pipeline([
    ('atributos_numericos', AtributosNumericos()),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

XnumTratado = pipenum.fit_transform(Xdrop)
XnumTratado

array([[ 0.82737724, -0.56573646,  0.43279337, -0.47367361, -0.50244517],
       [-1.56610693,  0.66386103,  0.43279337, -0.47367361,  0.78684529],
       [ 0.82737724, -0.25833709, -0.4745452 , -0.47367361, -0.48885426],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337,  2.00893337, -0.17626324],
       [-1.56610693, -0.25833709, -0.4745452 , -0.47367361, -0.04438104],
       [ 0.82737724,  0.20276197, -0.4745452 , -0.47367361, -0.49237783]])

In [28]:
class AtributosCategoricos(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self
  def transform(self, X, y=None):
    return X.select_dtypes(include='object')

atributos_categoricos = AtributosCategoricos()
Xcat = atributos_categoricos.fit_transform(Xdrop)
Xcat.head()

,Sex,Embarked
0,male,S
1,female,C
2,female,S
3,female,S
4,male,S


In [29]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

pipecat = Pipeline([
    ('atributos_categoricos', AtributosCategoricos()),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())
])

XcatTratado = pipecat.fit_transform(Xdrop)
XcatTratado

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1782 stored elements and shape (891, 5)>

In [30]:
from sklearn.pipeline import FeatureUnion

unecaracteristicas = FeatureUnion([
    ('pipenum', pipenum),
    ('pipecat', pipecat)
])

Xtratado = unecaracteristicas.fit_transform(Xdrop)
Xtratado

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6237 stored elements and shape (891, 10)>

In [31]:
preproc = Pipeline([
    ('atributos_desejados', AtributosDesejados()),
    ('unecaracteristicas', unecaracteristicas)
])

Xtratado = preproc.fit_transform(X)
Xtratado

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6237 stored elements and shape (891, 10)>

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

clf = Pipeline([
    ('preproc', preproc),
    ('classificador', RandomForestClassifier())
])

clf.fit(X, y)

Pipeline(steps=[('preproc',
                 Pipeline(steps=[('atributos_desejados', AtributosDesejados()),
                                 ('unecaracteristicas',
                                  FeatureUnion(transformer_list=[('pipenum',
                                                                  Pipeline(steps=[('atributos_numericos',
                                                                                   AtributosNumericos()),
                                                                                  ('imputer',
                                                                                   SimpleImputer(strategy='median')),
                                                                                  ('scaler',
                                                                                   StandardScaler())])),
                                                                 ('pipecat',
                                                                  Pipeline(steps=[('atributos_categoricos',
                                                                                   AtributosCategoricos()),
                                                                                  ('imputer',
                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                  ('encoder',
                                                                                   OneHotEncoder())]))]))])),
                ('classificador', RandomForestClassifier())])

In [33]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X)
accuracy_score(y, y_pred)

0.9797979797979798

In [35]:
from sklearn.model_selection import cross_validate
from pprint import pprint

scores = cross_validate(clf, X, y)
pprint(scores)
print(scores['test_score'].mean())

{'fit_time': array([0.47189617, 0.32359481, 0.46968532, 0.5319488 , 0.54626966]),
 'score_time': array([0.01931262, 0.01918864, 0.02985978, 0.0286274 , 0.02815819]),
 'test_score': array([0.77094972, 0.80337079, 0.85393258, 0.76966292, 0.84269663])}
0.808122528403741


In [36]:
clf.fit(X, y)
ypred = clf.predict(X)
ypred

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

In [39]:
agreed = y == ypred
agreed.value_counts()

,count
Survived,
True,873
False,18


In [40]:
test = pd.read_csv('test.csv')
test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [41]:
ypred = clf.predict(test)

In [42]:
submission = pd.read_csv('gender_submission.csv')
agreed = submission['Survived'] == ypred
agreed.value_counts()

,count
Survived,
True,345
False,73


In [43]:
submission['Survived'] = ypred
submission.to_csv('rf_basic.csv', index='False')

In [44]:
print(y.sum()/len(y))

0.3838383838383838


In [45]:
print(ypred.sum()/len(ypred))

0.3660287081339713


In [46]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(clf, X, y,
                        cv=StratifiedKFold(shuffle=True))
pprint(scores)
print(scores['test_score'].mean())

{'fit_time': array([0.38220334, 0.35998559, 0.57901216, 0.55022073, 0.55973053]),
 'score_time': array([0.01922345, 0.01996684, 0.02966762, 0.02818608, 0.22333956]),
 'test_score': array([0.84916201, 0.80337079, 0.81460674, 0.82022472, 0.78651685])}
0.814776222459356


In [51]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

parametros = {
    'preproc__unecaracteristicas__pipenum__imputer__strategy': ['mean', 'median'],
    'classificador__n_estimators': [20, 100, 200]
}

clf = Pipeline([
    ('preproc', preproc),
    ('classificador', RandomForestClassifier())
])

modelo = HalvingGridSearchCV(clf, param_grid=parametros)

scores = cross_validate(modelo, X, y,
                        cv=StratifiedKFold(shuffle=True))
pprint(scores)
print(scores['test_score'].mean())

{'fit_time': array([11.33109856, 11.41807008, 12.5827682 , 17.11947584, 14.43185592]),
 'score_time': array([0.01916218, 0.02116966, 0.01180053, 0.02950287, 0.02919126]),
 'test_score': array([0.79329609, 0.83707865, 0.8258427 , 0.78089888, 0.76404494])}
0.80023225158496


In [58]:
class DenseTransformer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.toarray()

preproc = Pipeline([
    ('atributos_desejados', AtributosDesejados()),
    ('unecaracteristicas', unecaracteristicas),
    ('to_dense', DenseTransformer())
])

Xtratado = preproc.fit_transform(X)
Xtratado

array([[ 0.82737724, -0.56573646,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [-1.56610693,  0.66386103,  0.43279337, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82737724, -0.25833709, -0.4745452 , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [-1.56610693, -0.25833709, -0.4745452 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82737724,  0.20276197, -0.4745452 , ...,  0.        ,
         1.        ,  0.        ]])

In [62]:
from sklearn.ensemble import StackingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier

import warnings
warnings.filterwarnings('ignore')

base_estimators = [
    ('rf', RandomForestClassifier()),
    ('et', ExtraTreesClassifier()),
    ('mlp', MLPClassifier()),
    ('gnb', GaussianNB()),
    ('knn', KNeighborsClassifier()),
    ('per', Perceptron()),
    ('lr', LogisticRegression()),
]

clf = StackingClassifier(
    estimators=base_estimators
)

model = Pipeline([
    ('preproc', preproc),
    ('classificador', clf)
])

scores = cross_validate(model, X, y,
                        cv=StratifiedKFold(shuffle=True))
pprint(scores)
print(scores['test_score'].mean())

{'fit_time': array([6.36741018, 5.0286386 , 6.40810061, 5.0869658 , 6.08554292]),
 'score_time': array([0.0477438 , 0.04800534, 0.04830956, 0.05021381, 0.07580185]),
 'test_score': array([0.77653631, 0.83146067, 0.84831461, 0.84831461, 0.8258427 ])}
0.826093779423765
